In [1]:
#V2 will check if the path is empty in the csv. 
#If empty and available fill it,not empty not rewrite it
#v3 do it withoyt RT STRuctBuilder
#v4 again with RTStruct Builder,  because v3 was incomplete, but now it does it with both ITV and GTVs labels separately

In [2]:
from rt_utils import RTStructBuilder
import matplotlib.pyplot as plt
import numpy as np
import pydicom
import os
import csv
import re
import pandas as pd

In [3]:
def FindCT_UID(path):
    files = os.listdir(path)
    ct_dicom_dataset = pydicom.dcmread(os.path.join(path,files[0]))
    series_instance_uid = ct_dicom_dataset[(0x0020, 0x0052)].value
    return series_instance_uid

In [4]:
def FindRT_UID_perContour(path,roi_name_to_find):
    files = os.listdir(path)
    rtstruct_dataset = pydicom.dcmread(path+files[0])
    if rtstruct_dataset.Modality == "RTSTRUCT":
        roi_sequence = rtstruct_dataset.StructureSetROISequence
        for roi_item in roi_sequence:
            roi_number = roi_item.ROINumber
            roi_name = roi_item.ROIName
            if roi_name == roi_name_to_find:
                # Print information for the found ROI
                #print(f"ROI Name: {roi_name}")
                #print(f"ROI NameTo find: {roi_name_to_find}")
                #print(f"ROI Number: {roi_number}")
                referenced_frame_of_reference_uid = roi_item.ReferencedFrameOfReferenceUID
                #print(f"Referenced Frame of Reference UID: {referenced_frame_of_reference_uid}")
                return referenced_frame_of_reference_uid

        else:
            return None


In [6]:
def LookForLabelMatch(ListOfNamesinRT,LabelsToLookFor,currPathTemp):
    tempList = []
    for labeltot in LabelsToLookFor:
        if labeltot in ListOfNamesinRT:
            tempList.append(currPathTemp)
            return tempList
    return tempList

In [24]:
def GetTheContourRTMatch(ctfolder,rtfolder,itv_tot_labels,itv_tumor_labels,itv_ln_labels,gtv_tot_labels,gtv_tumor_labels,gtv_ln_labels):
    singlematch = False # If one RT file has already been matched you dont need to loop to the next one. In theory only one RTStruct should be necessary
    ct_Match = []
    totGTV_Match, tumorGTV_Match, lnGTV_Match = ([],[],[])
    totITV_Match, tumorITV_Match, lnITV_Match = ([],[],[])
    gtv_bool = False
    itv_bool = False
    if len(ctfolder)==0:
        print("No CT")
    for k in range(len(ctfolder)):
        for j in range(len(rtfolder)):
            rtfiles = os.listdir(rtfolder[j])
            for i in range(len(rtfiles)):
                if not singlematch:
                    try:
                        print("check",i,j,k)
                        
                        rtstruct = RTStructBuilder.create_from(dicom_series_path=ctfolder[k],rt_struct_path=str(rtfolder[j]+rtfiles[i]))

                        totITV_Match = LookForLabelMatch(rtstruct.get_roi_names(),itv_tot_labels,str(rtfolder[j]+rtfiles[i]))
                        tumorITV_Match = LookForLabelMatch(rtstruct.get_roi_names(),itv_tumor_labels,str(rtfolder[j]+rtfiles[i]))
                        lnITV_Match = LookForLabelMatch(rtstruct.get_roi_names(),itv_ln_labels,str(rtfolder[j]+rtfiles[i]))

                        totGTV_Match = LookForLabelMatch(rtstruct.get_roi_names(),gtv_tot_labels,str(rtfolder[j]+rtfiles[i]))
                        tumorGTV_Match = LookForLabelMatch(rtstruct.get_roi_names(),gtv_tumor_labels,str(rtfolder[j]+rtfiles[i]))
                        lnGTV_Match = LookForLabelMatch(rtstruct.get_roi_names(),gtv_ln_labels,str(rtfolder[j]+rtfiles[i]))

                        if len(totITV_Match) ==0 and len(tumorITV_Match)+len(lnITV_Match) == 0:
                            print("No labels Matched for ITV",rtfiles[i],rtstruct.get_roi_names())
                        else: itv_bool = True
                        
                        if len(totGTV_Match) ==0 and len(tumorGTV_Match)+len(lnGTV_Match) == 0:
                            print("No labels Matched for GTV",rtfiles[i],rtstruct.get_roi_names())
                        else: gtv_bool = True
                        
                        if (len(totITV_Match)>0 or len(tumorITV_Match)+len(lnITV_Match)>0) or (len(totGTV_Match)>0 or len(tumorGTV_Match)+len(lnGTV_Match) >0):
                            ct_Match.append(ctfolder[k])
                            print("M ITV","tot",len(totITV_Match),"tumor",len(tumorITV_Match),"LN",len(lnITV_Match))
                            print("M GTV","tot",len(totGTV_Match),"tumor",len(tumorGTV_Match),"LN",len(lnGTV_Match))
                            if len(totITV_Match) == 0: totITV_Match.append('0')
                            if len(tumorITV_Match) == 0: tumorITV_Match.append('0')
                            if len(lnITV_Match) == 0: lnITV_Match.append('0')
                            if len(totGTV_Match) == 0: totGTV_Match.append('0')
                            if len(tumorGTV_Match) == 0: tumorGTV_Match.append('0')
                            if len(lnGTV_Match) == 0: lnGTV_Match.append('0')
                                
                        if gtv_bool and itv_bool:
                            singlematch = True
                    except Exception as e: print(e)#pass

        #mask_3d = rtstruct.get_roi_mask_by_name(label)
    return ct_Match,totITV_Match, tumorITV_Match, lnITV_Match,totGTV_Match, tumorGTV_Match, lnGTV_Match

In [25]:
#GET PATHS AND LABELS
root_path  = "//zkh/appdata/RTDicom/Projectline_modelling_lung_cancer/DICOM_data/DATA_VOLLEDIG_unstructured/"
#PxList = np.sort(os.listdir(root_path))
csv_file_path = "//zkh/appdata/RTDicom/Projectline_modelling_lung_cancer/Users/Luis/CSVs/CTandRTMatch_GTVandITV.csv"
# ITV Labels
itv_tot_labels = ["ITV", "IGTV", "IgTV", "ITV1", "ITV2", "ITV3", "ITV totaal def", "2ITV", "ITV_TOT", "ITV_6000", "ITV_5100", "ITV_Totaal", "ITV_LBK", "ITV_LOK", "IGTV_6000"]
itv_tumor_labels = ["ITVtumor", "ITV_tumor", "ITVtumor def", "2ITV_tumor", "ITV-P", "ITVtumorA1", "ITV_tumor_LBK", "ITVtu", "IGTVp"]
itv_ln_labels = ["ITVklieren", "ITV_klier", "ITV_Klier", "ITVklieren def", "2ITV_klier", "ITV_n", "ITV_klier_LBK", "IGTVnode"]

# GTV Labels
gtv_tot_labels = ["GTV", "GTV1", "GTV2", "GTV3", "GTV totaal def", "2GTV", "GTV_TOT", "GTV_6000", "GTV_5100", "GTV_Totaal", "GTV_LBK", "GTV_LOK", "GTV_5000", "GTV_preop", "GTV_totaal", "GTV_voor OK", "GTVtotaal", "GTVnew", "GTV(op exp)", "GTVop", "GTV_voor OK", "GTV oorspr"]
gtv_tumor_labels = ["GTVtumor", "GTV_tumor", "GTVtumor def", "2GTV_tumor", "GTV-P", "GTVtumorA1", "GTV_tumor_LBK", "GTV tumor", "GTVp", "GTVtu", "GTVtumor_exp", "GTVtumor_exp_50->0", "GTVtumor_insp", "GTVpatelectasebewaren(1)", "GTVt"]
gtv_ln_labels = ["GTVklieren", "GTV_klier", "GTV_Klier", "GTVklieren def", "2GTV_klier", "GTV_n", "GTV_klier_LBK", "GTVn", "GTVnode_exp", "GTVnodes_exp", "GTVnodes_exp_50->0", "GTVnodes_insp", "GTVklieren"]

fieldnames=['ID', 'CT', 'ITVTot', 'ITVTumor', 'ITVLN', 'GTVtot', 'GTVTumor', 'GTVLN']

PxListPath = '//zkh/appdata/RTDicom/Projectline_modelling_lung_cancer/Users/Luis/ListsOfPatients/MissingCTRTPx_Feb29.txt'
PxList = pd.read_csv(PxListPath,header=None,dtype=str)


In [28]:
#LOOK for patients already been analized
if os.path.exists(csv_file_path):
    df = pd.read_csv(csv_file_path)
    checked_list = df['ID'].tolist()
    print(len(checked_list),"Px already checked,",len(PxList),"Missing")
    

1027 Px already checked, 302 Missing


In [32]:
with open(csv_file_path, 'a', newline='') as csvfile:
    
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    if csvfile.tell() == 0:
        writer.writeheader()


    for Px in PxList[0]:
        strPx = str(Px)
        strPx = strPx.zfill(7)
        Px = strPx
        AvgCt_fpaths = []
        RTStruct_fpaths = []
        if not os.path.exists(csv_file_path):
            checked_list = []
        if int(Px) in checked_list:
            print(Px,"Px already checked")
        else:
            for root, dirs, files in os.walk(root_path+Px, topdown=True):
                for d in dirs:
                    if "Thorax" in d or "Ave" in d or "MIP" in d: #("CT" in d and not "RTSTRUCT" in d): #"%" in d or 
                        AvgCt_fpaths.append(root+'/'+d+'/')
                    if ('RTSTRUCT' in d or "pproved" in d) or ("PinnPlan" in d and not("RTDOSE" in d)):
                        RTStruct_fpaths.append(root+'/'+d+'/')


            print("Px",Px,"Num CTs",len(AvgCt_fpaths),"Num RTs",len(RTStruct_fpaths))

            ctMatch,rtMatch_ITVtot,rtMatch_ITVtumor,rtMatch_ITVln,rtMatch_GTVtot,rtMatch_GTVtumor,rtMatch_GTVln = GetTheContourRTMatch(AvgCt_fpaths,RTStruct_fpaths,itv_tot_labels,itv_tumor_labels,itv_ln_labels,gtv_tot_labels,gtv_tumor_labels,gtv_ln_labels)

            if len(ctMatch) == 0 :
                print("No Match")
            else: 
                print("Yes Match")
                
                writer.writerow({'ID':Px,'CT':ctMatch,'ITVTot':rtMatch_ITVtot,'ITVTumor':rtMatch_ITVtumor,'ITVLN':rtMatch_ITVln,'GTVtot':rtMatch_GTVtot,'GTVTumor':rtMatch_GTVtumor,'GTVLN':rtMatch_GTVln})


Px 3610529 Num CTs 2 Num RTs 1
check 0 0 0
No labels Matched for ITV IM1 (1).DCM ['CTV_6000', 'PTV_6000', 'verif_mask', 'IsocCT', 'Isoc N1F']
No labels Matched for GTV IM1 (1).DCM ['CTV_6000', 'PTV_6000', 'verif_mask', 'IsocCT', 'Isoc N1F']
check 1 0 0
Please check that the existing RTStruct is valid
check 2 0 0
No labels Matched for ITV IM1 (3).DCM ['CTV_6000', 'PTV_6000', 'verif_mask', 'IsocCT', 'Isoc N1F']
No labels Matched for GTV IM1 (3).DCM ['CTV_6000', 'PTV_6000', 'verif_mask', 'IsocCT', 'Isoc N1F']
check 3 0 0
Please check that the existing RTStruct is valid
check 0 0 1
Loaded RTStruct references image(s) that are not contained in input series data. Problematic image has SOP Instance Id: 1.2.840.113619.2.25.4.1433992366.3719.1559822041.1599
check 1 0 1
Please check that the existing RTStruct is valid
check 2 0 1
Loaded RTStruct references image(s) that are not contained in input series data. Problematic image has SOP Instance Id: 1.2.840.113619.2.25.4.1433992366.3719.1559822041

Px 8130104 Num CTs 1 Num RTs 5
check 0 0 0
Loaded RTStruct references image(s) that are not contained in input series data. Problematic image has SOP Instance Id: 1.3.12.2.1107.5.1.4.1005.30000012041308411667100009988
check 0 1 0
Please check that the existing RTStruct is valid
check 0 2 0
Loaded RTStruct references image(s) that are not contained in input series data. Problematic image has SOP Instance Id: 1.3.12.2.1107.5.1.4.1005.30000012041308411667100009988
check 0 3 0
Please check that the existing RTStruct is valid
check 0 4 0
Please check that the existing RTStruct is valid
No Match
Px 4156751 Num CTs 13 Num RTs 1
check 0 0 0
Loaded RTStruct references image(s) that are not contained in input series data. Problematic image has SOP Instance Id: 1.3.12.2.1107.5.1.4.95434.30000020032307562027100006390
check 1 0 0
Loaded RTStruct references image(s) that are not contained in input series data. Problematic image has SOP Instance Id: 1.3.12.2.1107.5.1.4.95434.3000002003230756202710000

Px 2159179 Num CTs 3 Num RTs 5
check 0 0 0
Loaded RTStruct references image(s) that are not contained in input series data. Problematic image has SOP Instance Id: 1.2.840.113619.2.278.3.663816714.946.1371209134.864.1
check 0 1 0
Please check that the existing RTStruct is valid
check 0 2 0
Please check that the existing RTStruct is valid
check 0 3 0
Loaded RTStruct references image(s) that are not contained in input series data. Problematic image has SOP Instance Id: 1.2.840.113619.2.278.3.663816714.946.1371209134.864.1
check 0 4 0
Please check that the existing RTStruct is valid
check 0 0 1
Loaded RTStruct references image(s) that are not contained in input series data. Problematic image has SOP Instance Id: 1.2.840.113619.2.278.3.663816714.946.1371209134.864.1
check 0 1 1
Please check that the existing RTStruct is valid
check 0 2 1
Please check that the existing RTStruct is valid
check 0 3 1
Loaded RTStruct references image(s) that are not contained in input series data. Problematic i

Px 1509318 Num CTs 2 Num RTs 1
check 0 0 0
No labels Matched for ITV 1.2.752.243.1.1.20181022114021328.1600.78633.DCM ['CTV_5100', 'PTV_5100', 'Verif_Mask', 'isocCT', 'IsocN1F']
No labels Matched for GTV 1.2.752.243.1.1.20181022114021328.1600.78633.DCM ['CTV_5100', 'PTV_5100', 'Verif_Mask', 'isocCT', 'IsocN1F']
check 1 0 0
No labels Matched for ITV 1.2.752.243.1.1.20181022141413506.3600.24201.DCM ['CTV_5100', 'PTV_5100', 'Verif_Mask', 'isocCT', 'IsocN1F']
No labels Matched for GTV 1.2.752.243.1.1.20181022141413506.3600.24201.DCM ['CTV_5100', 'PTV_5100', 'Verif_Mask', 'isocCT', 'IsocN1F']
check 0 0 1
Loaded RTStruct references image(s) that are not contained in input series data. Problematic image has SOP Instance Id: 1.3.12.2.1107.5.1.4.95434.30000018101606542918600008297
check 1 0 1
Loaded RTStruct references image(s) that are not contained in input series data. Problematic image has SOP Instance Id: 1.3.12.2.1107.5.1.4.95434.30000018101606542918600008297
No Match
9632494 Px already c

Loaded RTStruct references image(s) that are not contained in input series data. Problematic image has SOP Instance Id: 1.3.12.2.1107.5.1.4.95434.30000021012507185209300010083
check 0 0 13
Loaded RTStruct references image(s) that are not contained in input series data. Problematic image has SOP Instance Id: 1.3.12.2.1107.5.1.4.95434.30000021012507185209300010083
check 0 0 14
Loaded RTStruct references image(s) that are not contained in input series data. Problematic image has SOP Instance Id: 1.3.12.2.1107.5.1.4.95434.30000021012507185209300010083
check 0 0 15
Loaded RTStruct references image(s) that are not contained in input series data. Problematic image has SOP Instance Id: 1.3.12.2.1107.5.1.4.95434.30000021012507185209300010083
check 0 0 16
Loaded RTStruct references image(s) that are not contained in input series data. Problematic image has SOP Instance Id: 1.3.12.2.1107.5.1.4.95434.30000021012507185209300010083
check 0 0 17
Loaded RTStruct references image(s) that are not contai

...

Loaded RTStruct references image(s) that are not contained in input series data. Problematic image has SOP Instance Id: 1.3.12.2.1107.5.1.4.95434.30000022032906342636300011966
check 0 0 10
Loaded RTStruct references image(s) that are not contained in input series data. Problematic image has SOP Instance Id: 1.3.12.2.1107.5.1.4.95434.30000022032906342636300011966
check 0 0 11
Loaded RTStruct references image(s) that are not contained in input series data. Problematic image has SOP Instance Id: 1.3.12.2.1107.5.1.4.95434.30000022032906342636300011966
check 0 0 12
No labels Matched for ITV IM1.DCM ['CTV_6000', 'Verif_MaskP', 'Spinalcord+2mm', 'kVueProtonOne', 'QFixLongInsert', 'MonarchWings', 'Swelling+0.5', 'Swelling-0.5', 'BB', 'Air_In_Oes', 'CTV_6000+6mm', 'Wires', 'BP-A1RV-20p', 'BP-A1LA-20p', 'BP-A1RA-20p', 'isocCT', 'GA_CT3', 'TCP', 'MWP', 'IsocA1P']
No labels Matched for GTV IM1.DCM ['CTV_6000', 'Verif_MaskP', 'Spinalcord+2mm', 'kVueProtonOne', 'QFixLongInsert', 'MonarchWings', 'Swe

Loaded RTStruct references image(s) that are not contained in input series data. Problematic image has SOP Instance Id: 1.3.12.2.1107.5.1.4.95434.30000022081006482055200008659
check 0 0 8
Loaded RTStruct references image(s) that are not contained in input series data. Problematic image has SOP Instance Id: 1.3.12.2.1107.5.1.4.95434.30000022081006482055200008659
check 0 0 9
Loaded RTStruct references image(s) that are not contained in input series data. Problematic image has SOP Instance Id: 1.3.12.2.1107.5.1.4.95434.30000022081006482055200008659
check 0 0 10
No labels Matched for ITV IM1.DCM ['CTV_6000', 'PTV_6000', 'verif mask', 'isocCT', 'IsocA1F']
No labels Matched for GTV IM1.DCM ['CTV_6000', 'PTV_6000', 'verif mask', 'isocCT', 'IsocA1F']
check 0 0 11
Loaded RTStruct references image(s) that are not contained in input series data. Problematic image has SOP Instance Id: 1.3.12.2.1107.5.1.4.95434.30000022081006482055200008659
check 0 0 12
Loaded RTStruct references image(s) that are 

Px 6142030 Num CTs 14 Num RTs 0
No Match
2347347 Px already checked
2740939 Px already checked
2343354 Px already checked
4588342 Px already checked
1331234 Px already checked
3088232 Px already checked
3798445 Px already checked
3534670 Px already checked
Px 5369634 Num CTs 14 Num RTs 1
check 0 0 0
Loaded RTStruct references image(s) that are not contained in input series data. Problematic image has SOP Instance Id: 1.3.12.2.1107.5.1.4.95434.30000023010207573888900009352
check 0 0 1
Loaded RTStruct references image(s) that are not contained in input series data. Problematic image has SOP Instance Id: 1.3.12.2.1107.5.1.4.95434.30000023010207573888900009352
check 0 0 2
Loaded RTStruct references image(s) that are not contained in input series data. Problematic image has SOP Instance Id: 1.3.12.2.1107.5.1.4.95434.30000023010207573888900009352
check 0 0 3
Loaded RTStruct references image(s) that are not contained in input series data. Problematic image has SOP Instance Id: 1.3.12.2.1107.5

Loaded RTStruct references image(s) that are not contained in input series data. Problematic image has SOP Instance Id: 1.3.12.2.1107.5.1.4.95434.30000023051906245520500002847
check 0 0 13
Loaded RTStruct references image(s) that are not contained in input series data. Problematic image has SOP Instance Id: 1.3.12.2.1107.5.1.4.95434.30000023051906245520500002847
No Match
Px 7123039 Num CTs 14 Num RTs 2
check 0 0 0
Loaded RTStruct references image(s) that are not contained in input series data. Problematic image has SOP Instance Id: 1.3.12.2.1107.5.1.4.95434.30000023070506483792400000844
check 0 1 0
Loaded RTStruct references image(s) that are not contained in input series data. Problematic image has SOP Instance Id: 1.3.12.2.1107.5.1.4.95434.30000023070506483792400000844
check 0 0 1
Loaded RTStruct references image(s) that are not contained in input series data. Problematic image has SOP Instance Id: 1.3.12.2.1107.5.1.4.95434.30000023070506483792400000844
check 0 1 1
Loaded RTStruct re

Loaded RTStruct references image(s) that are not contained in input series data. Problematic image has SOP Instance Id: 1.3.12.2.1107.5.1.4.95434.30000023072106460312200000001
check 0 0 5
Loaded RTStruct references image(s) that are not contained in input series data. Problematic image has SOP Instance Id: 1.3.12.2.1107.5.1.4.95434.30000023072106460312200000001
check 0 0 6
Loaded RTStruct references image(s) that are not contained in input series data. Problematic image has SOP Instance Id: 1.3.12.2.1107.5.1.4.95434.30000023072106460312200000001
check 0 0 7
Loaded RTStruct references image(s) that are not contained in input series data. Problematic image has SOP Instance Id: 1.3.12.2.1107.5.1.4.95434.30000023072106460312200000001
check 0 0 8
Loaded RTStruct references image(s) that are not contained in input series data. Problematic image has SOP Instance Id: 1.3.12.2.1107.5.1.4.95434.30000023072106460312200000001
check 0 0 9
Loaded RTStruct references image(s) that are not contained i

In [31]:
RTStruct_fpaths

['//zkh/appdata/RTDicom/Projectline_modelling_lung_cancer/DICOM_data/DATA_VOLLEDIG_unstructured/3610529\\5679/RS Unapproved Structure Set/']

In [ ]:
ctfolder[k],str(rtfolder[j]+rtfiles[i])

In [ ]:
AvgCt_fpaths,RTStruct_fpaths